In [1]:
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
FILENAME = '/Users/shanewang/Desktop/6yue.xlsx'
SHEETNAME = '错号率-到项目'
sheetnames = list(pd.ExcelFile(FILENAME).sheet_names)
SHEETNAME = SHEETNAME if SHEETNAME in sheetnames else 0
print("Using sheet: ", SHEETNAME)
print("Sheet name: ", SHEETNAME if type(SHEETNAME) is str else sheetnames[0])
df = pd.read_excel(FILENAME, sheet_name=SHEETNAME) #一级考核-错号率
raw = df.copy(deep=True)
raw.head()

Using sheet:  0
Sheet name:  一级考核-错号率


,行标签,A1题甄别失败,S2题甄别失败,S4a题甄别失败,S4b题甄别失败,S6a题甄别失败,S6题甄别失败,S7题甄别失败,成功,关机/占线/无法接通/语音信箱,过滤问卷拒访,开场白拒访,空号/号码过期,磨合期-延期交付（未交付）,时间不宜,停机,未购买此楼盘,未接触,无此人/打错了,无人接听,语言不通,直接挂断/被访者不说话,重复访问,主体问卷拒访,准业主-提前交付（已交付）,接触量,错号数,接通数,错号率
0,集团,32.0,17.0,34.0,32.0,14.0,24.0,29.0,6088,4273.0,143.0,1419.0,166.0,1.0,1285.0,219.0,93.0,24059.0,122.0,10826.0,65.0,513.0,48.0,485.0,14.0,25942,617,10843,0.023784
1,上海区域,8.0,7.0,3.0,11.0,4.0,3.0,4.0,743,593.0,17.0,203.0,18.0,NaN,169.0,18.0,16.0,2026.0,13.0,1544.0,6.0,59.0,7.0,71.0,1.0,3518,72,1381,0.020466
2,蒙沪公司,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,16.0,NaN,7.0,NaN,NaN,8.0,NaN,NaN,110.0,NaN,54.0,NaN,1.0,NaN,1.0,NaN,113,0,43,0.000000
3,1-准业主1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,6.0,NaN,2.0,NaN,NaN,3.0,NaN,NaN,52.0,NaN,19.0,NaN,NaN,NaN,1.0,NaN,41,0,16,0.000000
4,2-准业主2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,10.0,NaN,3.0,NaN,NaN,5.0,NaN,NaN,58.0,NaN,31.0,NaN,1.0,NaN,NaN,NaN,65,0,24,0.000000


In [ ]:
def simplify(raw: pd.DataFrame):    
    labels = ['行标签', '总计', '接触量', '接触数','错号数', '错号量']
    for name in raw.columns.tolist():
        if name == '错号数':
            raw.rename(columns={'错号数':'错号量'}, inplace=True)
        if name == '接触数':
            raw.rename(columns={'接触数':'接触量'}, inplace=True)
        if name not in labels:
            raw.drop(name, axis=1, inplace=True)

        
simplify(raw)

raw.head(10)

In [ ]:
# raw.to_excel('sampleTest.xlsx', index=False)

In [ ]:
city_names = []
deprecated_names = ['集团', '上海区域']
# deprecated_names = ['集团']
for name in raw['行标签'].tolist():
    if name not in deprecated_names and '准业主' not in name \
                                    and '磨合期' not in name \
                                    and '老业主' not in name \
                                    and '稳定期' not in name:
        city_names.append(name)

print(city_names)
len(city_names)

In [ ]:
row_records = []
for _, row in raw.iterrows():
    try:
        row_records.append([row['行标签'], row['接触量'], row['错号量']])
    except Exception as e:
        print(e)
#         row_records.append([row['行标签'], row['总计'], row['错号量']])
    # print(row['行标签'], row['总计'], row['错号数'])
row_records

In [ ]:
wrong2city = {}
city_appeared = []
cities = []
slice_idx = []
zyz = []
mhq = []
wdq = []
lyz = []

for i in range(0, len(row_records)):
    tag = row_records[i][0]
    if tag in city_names:
        wrong2city[row_records[i][2]] = tag
        city_appeared.append(tag)
        slice_idx.append(i)
slice_idx.append(len(row_records))
for i in range(len(slice_idx)-1):
    cities.append(row_records[slice_idx[i]:slice_idx[i+1]])
print(cities[0])    
print(city_appeared)
print(slice_idx)

In [ ]:
for city_info in cities:
    CITY = city_info[0][0]
    for line in city_info[1:]:
        if '准业主' in line[0] and '准业主' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            zyz.append(line)
        if '磨合期' in line[0] and '磨合期' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            mhq.append(line)
        if '稳定期' in line[0] and '稳定期' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            wdq.append(line)
        if '老业主' in line[0] and '老业主' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            lyz.append(line)
zyz

In [ ]:
"""Get city info
    :return format: {CITY: [total, wrong]}
"""
def analyze(l: list):
    res = {}
    for i in range(len(l)):
        city = l[i][0].split('-')[0]
        if not res.__contains__(city):
            res[city] = [l[i][1], l[i][2]]
        else:
            res[city][0] += l[i][1]
            res[city][1] += l[i][2]
    return res

"""Calculate wrong rate
    :return format: {CITY: [total, wrong, wrong rate]}
"""
def calculate(d: dict):
    for k in d.keys():
        print(k, d[k])
        d[k].append(float('{:.5f}'.format(d[k][1]/d[k][0])))
    return d

In [ ]:
zyz_list = []
mhq_list = []
wdq_list = []
lyz_list = []
for k, v in calculate(analyze(zyz)).items():
    zyz_list.append(tuple([k, '准业主'] + v))
for k, v in calculate(analyze(mhq)).items():
    mhq_list.append(tuple([k, '磨合期'] + v))
for k, v in calculate(analyze(wdq)).items():
    wdq_list.append(tuple([k, '稳定期'] + v))
for k, v in calculate(analyze(lyz)).items():
    lyz_list.append(tuple([k, '老业主'] + v))

In [ ]:
labels = ['城市', '业主阶段', '接触量', '错号量', '错号率']
zyz_xlxs = pd.DataFrame(data=zyz_list, columns=labels)
zyz_xlxs = zyz_xlxs.sort_values(by='错号率', ascending=False)
zyz_xlxs.head(20)

In [ ]:
mhq_xlxs = pd.DataFrame(data=mhq_list, columns=labels)
mhq_xlxs = mhq_xlxs.sort_values(by='错号率', ascending=False)
mhq_xlxs.head(20)

In [ ]:
wdq_xlxs = pd.DataFrame(data=wdq_list, columns=labels)
wdq_xlxs = wdq_xlxs.sort_values(by='错号率', ascending=False)
wdq_xlxs.head(20)

In [ ]:
lyz_xlxs = pd.DataFrame(data=lyz_list, columns=labels)
lyz_xlxs = lyz_xlxs.sort_values(by='错号率', ascending=False)
lyz_xlxs.head(20)

In [ ]:
# Shanghai
def getShanghaiInfo(raw, zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs):
    for _, rows in raw.iterrows():
        if '准业主' in rows['行标签']:
            # tuple = ('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量'])))
            # print(('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
            shzyz = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['准业主'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            zyz_xlxs = pd.concat([zyz_xlxs, shzyz])

        if '磨合期' in rows['行标签']:
            shmhq = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['磨合期'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})

            mhq_xlxs = pd.concat([mhq_xlxs, shmhq])
        if '稳定期' in rows['行标签']:
            shwdq = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['稳定期'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            wdq_xlxs = pd.concat([wdq_xlxs, shwdq])
        if '老业主' in rows['行标签']:
            shlyz = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['老业主'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            lyz_xlxs = pd.concat([lyz_xlxs, shlyz])
    return [zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs]

dflist = [zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs]
try:
    df = pd.read_excel(FILENAME, sheet_name='上海区域')
    raw = df.copy(deep=True)
    simplify(raw)
    dflist = getShanghaiInfo(raw, zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs)
except Exception as e:
    print(e)
    # print("No such sheet!")
    pass

In [ ]:
# Excel weiter funtion
def multiple_dfs(df_list, sheets, file_name, spaces):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    row = 0
    for dataframe in df_list:
        dataframe.to_excel(writer, sheet_name=sheets, startrow=row, startcol=0, index=False)   
        row = row + len(dataframe.index) + spaces + 1
    writer.save()

# write list of dataframes
import time
file_name = FILENAME.strip().split('/')[-1].strip().split('.')[0]+'-'+time.strftime("%Y%m%d", time.localtime())+'.xlsx'
multiple_dfs(dflist, 'Validation', file_name, 1)
print("File processed completed!")